In [3]:
import requests, json
from dataclasses import dataclass, asdict

coins = { "BTC", "BCH", "DOGE", "LTC", "KAS", "RVN"}
api_token = "8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92"
coins_endpoint = f"https://whattomine.com/api/v1/coins?api_token={api_token}"
json = requests.get(coins_endpoint).json()

@dataclass
class WhatToMineData:
    tag: str = ""
    block_time: float = 0
    block_reward_btc: float = 0
    difficulty: float = 0
    nethash: float = 0
    daily_emissions_btc: float = 0
    algorithm: str = ""
    exchange_rate_btc: float = 0

def toWhatToMineData(json):
    data = WhatToMineData()
    data.tag = json['tag']
    data.block_time = float(json['block_time'])
    data.exchange_rate_btc = 1 if data.tag == "BTC" else json['exchanges'][0]['price']
    data.block_reward_btc = json['block_reward'] * data.exchange_rate_btc
    data.difficulty = json['difficulty']
    data.nethash = json['nethash']
    data.daily_emissions_btc = 86400 / data.block_time * data.block_reward_btc
    data.algorithm = json['algorithm']
    return data

coins_data = [ toWhatToMineData(coin) for coin in json if coin['tag'] in coins ]
print("Global coins data:\n")
for coin_data in coins_data:
    print(asdict(coin_data))


Global coins data:

{'tag': 'BCH', 'block_time': 615.0, 'block_reward_btc': 0.010641938843239562, 'difficulty': 375354846492.4, 'nethash': 2621360634276354471, 'daily_emissions_btc': 1.4950626277331678, 'algorithm': 'SHA-256', 'exchange_rate_btc': 0.00340542042983666}
{'tag': 'BTC', 'block_time': 656.0, 'block_reward_btc': 3.15763051, 'difficulty': 108105433845147.0, 'nethash': 707788571470728506097, 'daily_emissions_btc': 415.88304278048787, 'algorithm': 'SHA-256', 'exchange_rate_btc': 1}
{'tag': 'DOGE', 'block_time': 63.0, 'block_reward_btc': 0.0271, 'difficulty': 30700998.25649904, 'nethash': 2093012435971688, 'daily_emissions_btc': 37.16571428571428, 'algorithm': 'Scrypt', 'exchange_rate_btc': 2.71e-06}
{'tag': 'KAS', 'block_time': 0.9988, 'block_reward_btc': 6.545527811985209e-05, 'difficulty': 332763872.3570478, 'nethash': 1430927061537683219, 'daily_emissions_btc': 5.662130586258731, 'algorithm': 'kHeavyHash', 'exchange_rate_btc': 1.0005050342678568e-06}
{'tag': 'LTC', 'block_ti

In [2]:
import requests, json
import pandas as pd

algorithms = { "SCRYPT", "SHA256ASICBOOST", "KHEAVYHASH", "KAWPOW"}

multialgo_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = multialgo_response.json()
algo_data = { element['algorithm']: {'paying': element['paying'], 'speed': element['speed']} for element in multialgo_data['miningAlgorithms'] if element['algorithm'] in algorithms }

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
algorithms_response.raise_for_status()
algorithms_data = algorithms_response.json()

index = [ element['algorithm'] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]
columns = ['marketFactor', 'displayMarketFactor', 'priceFactor', 'displayPriceFactor', 'miningFactor', 'displayMiningFactor', 'priceDownStep', 'paying', 'speed']
data = [ [ float(element['marketFactor']), element['displayMarketFactor'], float(element['priceFactor']), element['displayPriceFactor'], float(element['miningFactor']), element['displayMiningFactor'], float(element['priceDownStep']), algo_data[element['algorithm']]['paying'], algo_data[element['algorithm']]['speed'] ] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]

raw_data_df = pd.DataFrame(data, index=index, columns=columns)

calc_df = pd.DataFrame(index=index)
sats2btc = 1e8

calc_df['hashrate'] = raw_data_df['speed'] / raw_data_df['marketFactor'] * raw_data_df['miningFactor']
calc_df['hashrate unit'] = raw_data_df['displayMarketFactor'] + '/s'

calc_df['paying'] = raw_data_df['paying'] * raw_data_df['priceFactor'] / sats2btc
calc_df['paying unit'] = 'BTC/' + raw_data_df['displayPriceFactor'] + '/day'

calc_df['daily_consumption_capacity'] = calc_df['hashrate'] * calc_df['paying']
calc_df['daily_consumption_capacity unit'] = 'BTC/day'

print("Nicehash data:\n")
json.loads(calc_df.to_json(orient='index'))


Nicehash data:



{'SCRYPT': {'hashrate': 90.3741677064,
  'hashrate unit': 'TH/s',
  'paying': 0.0188176301,
  'paying unit': 'BTC/TH/day',
  'daily_consumption_capacity': 1.70062766,
  'daily_consumption_capacity unit': 'BTC/day'},
 'SHA256ASICBOOST': {'hashrate': 6.7502196247,
  'hashrate unit': 'EH/s',
  'paying': 0.5690439857,
  'paying unit': 'BTC/EH/day',
  'daily_consumption_capacity': 3.8411718797,
  'daily_consumption_capacity unit': 'BTC/day'},
 'KAWPOW': {'hashrate': 0.770417096,
  'hashrate unit': 'TH/s',
  'paying': 0.0773410241,
  'paying unit': 'BTC/TH/day',
  'daily_consumption_capacity': 0.0595848472,
  'daily_consumption_capacity unit': 'BTC/day'},
 'KHEAVYHASH': {'hashrate': 55.7007459577,
  'hashrate unit': 'PH/s',
  'paying': 0.0039135099,
  'paying unit': 'BTC/PH/day',
  'daily_consumption_capacity': 0.217985423,
  'daily_consumption_capacity unit': 'BTC/day'}}